In [1]:
import torch
from utils.base_model import BiLSTM_CRF
from utils.utils import prepare, predict, train, evaluate

hparams = {
    'path':'data/data.txt',
    'epochs': 80,
    'batch_size': 100,
    'embedding_dim': 300,
    'hidden_dim': 256,
    'device':'cuda:1',
    'seq_length': 256,
    'save_path':'model_params/base_model.model'
}

In [2]:
attr_dict, loaders = prepare(hparams)
hparams['vocab_size'] = len(attr_dict['tokenizer'])

In [3]:
base_model = BiLSTM_CRF(hparams, attr_dict['tag2idx']).to(hparams['device'])

In [4]:
base_model = train(hparams, base_model, loaders)

epoch 1 , step 8 , loss: 447.5368: : 9it [00:02,  3.62it/s]
{'macro_f1': 0.0768, 'weighted_f1': 0.7796}
epoch 2 , step 8 , loss: 270.1000: : 9it [00:02,  3.51it/s]
{'macro_f1': 0.0765, 'weighted_f1': 0.7771}
epoch 3 , step 8 , loss: 239.7688: : 9it [00:02,  3.66it/s]
{'macro_f1': 0.0778, 'weighted_f1': 0.7823}
epoch 4 , step 8 , loss: 199.2411: : 9it [00:02,  3.45it/s]
{'macro_f1': 0.1167, 'weighted_f1': 0.8271}
epoch 5 , step 8 , loss: 170.4554: : 9it [00:02,  3.68it/s]
{'macro_f1': 0.1421, 'weighted_f1': 0.8413}
epoch 6 , step 8 , loss: 152.2573: : 9it [00:02,  3.29it/s]
{'macro_f1': 0.1614, 'weighted_f1': 0.8503}
epoch 7 , step 8 , loss: 139.1526: : 9it [00:02,  3.37it/s]
{'macro_f1': 0.1754, 'weighted_f1': 0.8556}
epoch 8 , step 8 , loss: 129.6469: : 9it [00:02,  3.33it/s]
{'macro_f1': 0.2035, 'weighted_f1': 0.8595}
epoch 9 , step 8 , loss: 120.0105: : 9it [00:02,  3.38it/s]
{'macro_f1': 0.2314, 'weighted_f1': 0.8659}
epoch 10 , step 8 , loss: 110.3648: : 9it [00:02,  3.62it/s]
{'m

In [ ]:
evaluate(base_model, loaders[1])

In [5]:
predict(['中国人民大学第三十五届一二九合唱音乐节如期举行，信息学院分团委文化部将组织信院全体同学参加'], base_model, attr_dict['tokenizer'])

[['B-department',
  'I-department',
  'I-department',
  'I-department',
  'I-department',
  'I-department',
  'O',
  'O',
  'I-scholarship',
  'I-scholarship',
  'I-scholarship',
  'I-scholarship',
  'I-scholarship',
  'I-scholarship',
  'I-scholarship',
  'I-scholarship',
  'I-scholarship',
  'I-scholarship',
  'I-scholarship',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-department',
  'I-department',
  'I-department',
  'I-organization',
  'I-organization',
  'I-organization',
  'I-organization',
  'I-organization',
  'I-organization',
  'I-organization',
  'O',
  'O',
  'O',
  'I-organization',
  'I-department',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  '[PAD]',
  '[PAD]',
  '[PAD]',
  '[PAD]',
  '[PAD]',
  '[PAD]',
  '[PAD]',
  '[PAD]',
  '[PAD]',
  '[PAD]',
  '[PAD]',
  '[PAD]',
  '[PAD]',
  '[PAD]',
  '[PAD]',
  '[PAD]',
  '[PAD]',
  '[PAD]',
  '[PAD]',
  '[PAD]',
  '[PAD]',
  '[PAD]',
  '[PAD]',
  '[PAD]',
  '[PAD]',
  '[PAD]',
  '[PAD]',
  '[PAD]',
  '[PAD]',
  '[PAD]',
  '[PAD]'

In [13]:
torch.save(base_model.state_dict(), hparams['save_path'])